In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import napari
from PIL import Image

from scribbles_creator import *
from cellpose_data_handler import *

c:\Users\roman\anaconda3\envs\ilastik\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prediction

Define prediction parameters

In [3]:
# Where to find and save the data
# folder_path = "./cellpose_results/data/run02_examples"
folder_path = "./cellpose_train_imgs"

mode = "all"
bins = [0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1]
scribble_width = None
# all_suff = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
# suff = all_suff[:1]
s = "a"

# Define the convpaint parameters
convpaint_layers = [[0], [0,2], [0,2,5]] # layers to use for convpaint (depending on the model chosen)
convpaint_scalings = [[1], [1,2], [1,2,4], [1,2,4,8]] # scalings for convpaint (downscaling the image)
model="vgg16" # 'vgg16', 'efficient_netb0', 'single_layer_vgg16', 'single_layer_vgg16_rgb', 'dino_vits16'
use_ilastik = True
use_dino = True
pred_seed = 123 # seed used for random forest classifier

save_res = False
show_res = True

In [ ]:
for bin in [0.1]: #bins:
    for img_num in range(120, 131, 20):#range(0, 10):
        print(f"IMG {img_num}: bin {bin}, suff {s}")

        for layers in [[0]]:#convpaint_layers:
            for scalings in [[1,2]]:#convpaint_scalings:
                print(f"   ConvPaint: layers {layers}, scalings {scalings}")
                pred_conv = pred_cellpose_convpaint(folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s,
                                                    layer_list=layers, scalings=scalings, model=model, random_state=pred_seed,
                                                    save_res=save_res, show_res=show_res)
        if use_ilastik:
            print("   Ilastik")
            pred_ila = pred_cellpose_ilastik(folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s,
                                                 random_state=pred_seed,
                                                 save_res=save_res, show_res=show_res)
        if use_dino:
            print("   DINO")
            pred_dino = pred_cellpose_dino(folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s,
                                           dinov2_model='s', upscale_order=0, random_state=pred_seed,
                                           save_res=save_res, show_res=show_res)
        print("\n")

Show the last predictions (optionally with image, ground truth and scribbles)

In [ ]:
img_data = get_cellpose_img_data(folder_path, img_num, load_img=True, load_gt=True, load_scribbles=True, mode=mode, bin=bin, scribble_width=scribble_width, suff=s)
img = img_data["img"]
ground_truth = img_data["gt"]
scribbles = img_data["scribbles"]

In [ ]:
print(f"Ilastik:   {np.sum(pred_ila == ground_truth) / np.prod(pred_ila.shape) * 100:.2f}%")
print(f"Convpaint: {np.sum(pred_conv == ground_truth) / np.prod(pred_conv.shape) * 100:.2f}%")
print(f"DINOv2:    {np.sum(pred_dino == ground_truth) / np.prod(pred_dino.shape) * 100:.2f}%")

In [ ]:
v = napari.Viewer()
v.add_image(img)
v.add_labels(ground_truth)
v.add_labels(scribbles)
v.add_labels(pred_conv)
v.add_labels(pred_ila)
v.add_labels(pred_dino)

## Test Computation Times

In [4]:
res_list = []
for img_num in range(0, 540, 100):
    labels = get_cellpose_img_data(folder_path, img_num, load_img=False, load_gt=False, load_scribbles=True, mode="all", bin=0.1, scribble_width=None, suff="a", load_pred=False, pred_tag="convpaint")["scribbles"]
    print(f"IMG {img_num}:")
    for layers in convpaint_layers:
        for scalings in convpaint_scalings:
            pred_tag = generate_convpaint_tag(layers, scalings, model)
            print(pred_tag)
            t = time_cellpose(img_num, folder_path, labels, pred_type="convpaint", layer_list=layers, scalings=scalings, model_name=model)
            res = pd.DataFrame({"img": img_num, "prediction tag": pred_tag, "prediction type": "convpaint",
                                "layers": str(layers)[1:-1].replace(", ", "-"), "scalings": str(scalings)[1:-1].replace(", ", "-"),
                                "t_features": t[0], "t_pred": t[1], "t_tot": t[2]}, index=[0])
            res_list.append(res)
    if use_ilastik:
        print("ilastik")
        t = time_cellpose(img_num, folder_path, labels, pred_type="ilastik")
        res = pd.DataFrame({"img": img_num, "prediction tag": "ilastik", "prediction type": "classical filters",
                            "layers": "-", "scalings": "-",
                            "t_features": t[0], "t_pred": t[1], "t_tot": t[2]}, index=[0])
        res_list.append(res)
    if use_dino:
        print("DINOv2")
        t = time_cellpose(img_num, folder_path, labels, pred_type="dino")
        res = pd.DataFrame({"img": img_num, "prediction tag": "dino", "prediction type": "DINOv2",
                            "layers": "-", "scalings": "-",
                            "t_features": t[0], "t_pred": t[1], "t_tot": t[2]}, index=[0])
        res_list.append(res)

df = pd.concat(res_list)

IMG 0:
convpaint_l-0_s-1
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 383, 512)
convpaint_l-0_s-1-2
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 383, 512)
convpaint_l-0_s-1-2-4
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 383, 512)
convpaint_l-0_s-1-2-4-8
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 383, 512)
convpaint_l-0-2_s-1
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 383, 512)
convpaint_l-0-2_s-1-2
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 383, 512)
convpaint_l-0-2_s-1-2-4
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 383, 512)
convpaint_l-0-2_s-1-2-4-8
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 383, 512)
convpaint_l-0-2-5_s-1
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) -

C:\Users\roman/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\roman/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\roman/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


IMG 100:
convpaint_l-0_s-1
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 450, 450)
convpaint_l-0_s-1-2
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 450, 450)
convpaint_l-0_s-1-2-4
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 450, 450)
convpaint_l-0_s-1-2-4-8
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 450, 450)
convpaint_l-0-2_s-1
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 450, 450)
convpaint_l-0-2_s-1-2
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 450, 450)
convpaint_l-0-2_s-1-2-4
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 450, 450)
convpaint_l-0-2_s-1-2-4-8
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 450, 450)
convpaint_l-0-2-5_s-1
Active channels: R=True, G=True, B=False --> Removed 1 channel(s)

In [5]:
df.groupby(["prediction type", "scalings", "layers"]).mean(True)[["t_features", "t_pred", "t_tot"]].round(2)

t_features  t_pred  t_tot
prediction type   scalings layers                           
DINOv2            -        -             2.20    0.48   2.67
classical filters -        -             0.38    0.81   1.20
convpaint         1        0             0.04    1.22   1.38
                           0-2           0.22    1.22   1.63
                           0-2-5         0.48    1.82   2.92
                  1-2      0             0.06    1.35   1.69
                           0-2           0.33    1.61   2.48
                           0-2-5         0.64    2.32   4.06
                  1-2-4    0             0.10    1.32   1.81
                           0-2           0.42    2.06   3.34
                           0-2-5         0.72    2.89   5.48
                  1-2-4-8  0             0.11    1.27   1.85
                           0-2           0.44    2.36   4.04
                           0-2-5         0.86    4.50   7.38

In [6]:
# df.to_csv(f"./cellpose_time_prelim1.csv", index=False)